This script check SE difference of control and exposed group at each clsuter/super-cluter.

In [1]:
# This script check SE difference of control and exposed group at each clsuter/super-cluter.
import sys
sys.path.append('../src')
import os
import se
import copy
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
import tree_util
import seaborn as sns
from statsmodels.stats.multitest import fdrcorrection as fdr
import warnings
warnings.filterwarnings("ignore")

In [2]:
# compute FR network
def fr(d_df, profile, sname):
    sp_list = list(profile.columns)
    sp_d_df = d_df.loc[sp_list, sp_list]
    #print(profile)
    sp_profile = np.array(profile.loc[sname, sp_list])
    value = np.dot(sp_profile.reshape(len(sp_profile), 1),sp_profile.reshape(1, len(sp_profile)))
    width = value.shape[0]
    cor_df = np.ones(shape=(width, width)) - sp_d_df.values
    for i in range(width):
        cor_df[i][i] = 0
    value = np.multiply(value, cor_df)
    df = pd.DataFrame(data= value, index=sp_list, columns=sp_list)
    return df

# compute SE for all sample
def multisample_se(profile, d_df, parent_dict, node_leaves, child_dict, direct_children_dict, param):
    result = pd.DataFrame(index=profile.index, columns=(list(node_leaves.keys()) + ['nroot']))
    for sname in profile.index:
       # print(sname)
       edge_df = fr(d_df, profile, sname)
       tmp = se.subtree_se_adj(edge_df, parent_dict, node_leaves, child_dict, direct_children_dict, param)
       for node in tmp.keys():
           value = tmp[node]
           result.loc[sname, node] = value
    if 'nroot' in result.columns:
        result.drop('nroot', axis=1, inplace=True)
    return result


In [3]:
outdir = '../result/Anti/SE'
if not os.path.exists(outdir):
    os.makedirs(outdir)

# load tree and construct tree with newick string
with open('../result/GCN_fix_tree/renamed_GCN_tree.newick') as fp:
    newick_tree = fp.read()
    # newick_tree = newick_tree.replace('-', '_')

json_tree = tree_util.parse(newick_tree)
largest = {'largest': 0}
leaf_list, l = tree_util.recu_compute(json_tree, 0, largest)
largest_level = largest['largest']
nlayer = largest_level
layer_leaves_dict = tree_util.make_layer_dict(nlayer)
tree_util.recu_layer(json_tree, layer_leaves_dict)
tree_util.to_layer_leaves(layer_leaves_dict, nlayer)
leaves_dict = copy.deepcopy(layer_leaves_dict)
parent_dict = {}
tree_util.parents(json_tree, parent_dict)
node_leaves = {}
for level in layer_leaves_dict.keys():
    for node, sp_list in layer_leaves_dict[level].items():
        if node in node_leaves.keys():
            continue
        node_leaves[node] = copy.deepcopy(sp_list)
subtree_nodes = {}
for l in leaf_list:
    parent = parent_dict[l]
    if parent not in subtree_nodes.keys():
        subtree_nodes[parent] = []
    subtree_nodes[parent].append(l)

for node in node_leaves.keys():
    parent = parent_dict[node]
    if parent not in subtree_nodes.keys():
        subtree_nodes[parent] = []
    subtree_nodes[parent] += subtree_nodes[node]
    subtree_nodes[parent].append(node)

for node in subtree_nodes.keys():
    subtree_nodes[node].append(node)

direct_children_dict = {}
for node, parent in parent_dict.items():
    if parent not in direct_children_dict:
        direct_children_dict[parent] = []
    direct_children_dict[parent].append(node)

node_leaves['root'] = list(leaf_list)

In [4]:
param = 0.5
p_cutoff = 0.05

# load distance, abundance profile and metadata
d_df = pd.read_csv('../data/sp_d.tsv', sep='\t', header=0, index_col=0)
metadata = pd.read_csv('../data/Anti/metadata.csv', sep=',', index_col=None, header=0)
abd = pd.read_csv('../data/Anti/abd.tsv', sep='\t', header=0, index_col=0)
metadata = metadata[['Sample_Name', 'Timepoint', 'type']]
metadata = metadata.drop_duplicates(inplace=False)
sname_tmp = 'P{}E{}'
abd_name_dict = {}
for idx in abd.index:
    abd_name_dict[idx] = idx.split('|')[-1].replace('_', '-')
abd = abd.rename(index=abd_name_dict)    
name_dict = {}
for sp in d_df.columns:
        name_dict[sp] = sp.replace('_', '-')
d_df = d_df.rename(columns=name_dict, index=name_dict)
abd = abd.T/100
sub_ids = sorted(list(set(metadata['Sample_Name'])))
days = sorted(list(set(metadata['Timepoint'])))
phenos = list(set(metadata['type']))
profile = abd[list(set(abd.columns).intersection(set(d_df.columns)))]

# compute SE for all samples
se_result = multisample_se(profile, d_df, parent_dict, node_leaves, subtree_nodes, direct_children_dict, param)
se_result.to_csv(os.path.join(outdir, 'se_df.tsv'), sep='\t')

In [5]:
if not os.path.exists(outdir):
    os.makedirs(outdir)

p1 = phenos[1]
p0 = phenos[0]

# differential test
p_df = pd.DataFrame(columns=days, index=list(node_leaves.keys()))
valid_dict = {}
for k in node_leaves.keys():
    metadata[k] = list(se_result.loc[list(metadata['Sample_Name']), k] )
    for j in days:
        selected0 = list(metadata[(metadata['Timepoint'] == j) & (metadata['type'] == p0)]['Sample_Name'])
        selected1 = list(metadata[(metadata['Timepoint'] == j) & (metadata['type'] == p1)]['Sample_Name'])
        selected_v0 = se_result.loc[selected0, k]
        selected_v1 = se_result.loc[selected1, k]
        if (list(selected_v0).count(0) > len(list(selected_v0))*0.8) and (list(selected_v1).count(0) > len(list(selected_v1))*0.8):
            continue
        if j not in valid_dict.keys():
            valid_dict[j] = []
        valid_dict[j].append(k)
        t, p = mannwhitneyu(list(selected_v0), list(selected_v1))
        p_df.loc[k, j] = p

# adjust with FDR  
p_df_adj = pd.DataFrame(columns=p_df.columns, index=list(node_leaves.keys()))
for d in days:
    p_values = p_df.loc[valid_dict[d], d]
    p_adj = fdr(p_values, p_cutoff)[1]
    for i, cluster in enumerate(valid_dict[d]):
        p_df_adj.loc[cluster, d] = p_adj[i]
p_df_adj.fillna(np.nan, inplace=True)
p_df_adj.to_csv(os.path.join(outdir, 'p_value.tsv'), sep='\t')

In [6]:
# make boxplot
for k in node_leaves.keys():
    metadata[k] = list(se_result.loc[list(metadata['Sample_Name']), k] )
    max_v = max(list(metadata[k]))
    show = False
    for j in days:
        if k in valid_dict[j]:
            show = True
    if not show:
        continue
    sns.boxplot(x='Timepoint', y=k, hue='type', data=metadata)
    for i, t in enumerate(days):
        if pd.isna(p_df_adj.loc[k, t]):
            plt.text(i, max_v*1.2, 'X', ha='center', va='bottom', fontsize=20, color='black')

        if p_df_adj.loc[k, t] < p_cutoff:
            plt.text(i, max_v*1.2, '**', ha='center', va='bottom', fontsize=20, color='r')
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.ylim(-0.1*max_v, max_v*1.5)
    plt.title(k)
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, '{}.pdf'.format(k)), dpi=300, format='pdf')
    # plt.show()
    plt.clf()
